## Lab 8: Текстурный анализ и контрастирование
### Вариант 7

In [28]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image as pim, ImageDraw
from tqdm import tqdm

# PARAMETERS
WHITE = 255
D = 2
PHI = [angle / 180 * np.pi for angle in [0, 90, 180, 270]]


In [29]:
def analyze_adjacent_pixels(input_image, pix, pos):
    output = np.zeros(WHITE + 1)
    base_x, base_y = pos
    for angle in PHI:
        x = base_x + np.around(np.cos(angle)) * D
        y = base_y + np.around(np.sin(angle)) * D
        if 0 <= x < input_image.size[0] and 0 <= y < input_image.size[1]:
            val = pix[x, y]
            output[val] += 1
            
    return pix[base_x, base_y], output

In [30]:
def pixel_gen(input_image, func=lambda input_image, pix, x: pix[x]):
    pix = input_image.load()
    for row in range(input_image.size[1]):
        for col in range(input_image.size[0]):
            pos = (col, row)
            yield pos, func(input_image, pix, pos)

In [31]:
def create_histogram(hist, img_name):
    f = plt.figure()
    plt.bar(np.arange(hist.size), hist)
    plt.savefig(f"{img_name}_bar.png")
    plt.close(f)

In [32]:
def calc_params(h_image, filename):
    res_s = pd.Series({"con": 0, "lun": 0})
    for (i, j), p in tqdm(pixel_gen(h_image), total=h_image.size[0] * h_image.size[1]):
        tmp = (i - j) ** 2
        res_s["con"] += tmp * p
        res_s["lun"] += p / (1 + tmp)
    res_s.to_csv(f"{filename}.csv")

In [37]:
def create_haralic_matrix_and_stats(name, mono_image):
    result = np.zeros((WHITE + 1, WHITE + 1))
    hist = np.zeros(WHITE + 1)
    max_max = 0
    total_pixels = mono_image.size[0] * mono_image.size[1]
    for pos, (val, row) in tqdm(pixel_gen(mono_image, func=analyze_adjacent_pixels), total=total_pixels):
        result[val] += row
        max_max = max(max_max, max(row))
        hist[val] += 1
    result_image = pim.fromarray(np.uint8(result * WHITE / max_max))
    result_image.save(f"{name}_matrix.png", "PNG")

    create_histogram(hist, name)
    calc_params(result_image, name)

In [34]:
def transform(image, c=1, f0=0, y=0.5):
    result_image = image.copy()
    d = ImageDraw.Draw(result_image)
    for pos, pixel in pixel_gen(image):
        p = min(int(WHITE * c * (pixel / WHITE + f0) ** y), WHITE)
        d.point(pos, p)
        
    return result_image

#### Main

In [41]:
def main():
    images = ["im1.png"]

    for image in images:
        output_path = "output/" + image
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        mono_img = pim.open("input/" + image).convert('L')
        mono_img.save(output_path + "/" + "mono.png", "PNG")

        create_haralic_matrix_and_stats(output_path + "/" + "mono", mono_img)

        log_img = transform(mono_img)
        log_img.save(output_path + "/" + "log.png", "PNG")
        create_haralic_matrix_and_stats(output_path + "/" + "log", log_img)

if __name__ == "__main__":
    main()


100%|██████████| 65536/65536 [00:01<00:00, 64048.41it/s]
